In [1]:
from dotenv import load_dotenv
import torch
import google.generativeai as genai
import os
import json

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Em nhờ chatgpt generate ra mẫu dữ liệu và em chỉ giả định database là 1 python dict chứ thật ra phải là MongoDB database vì vấn đề thời gian.

In [61]:
shipping_data = [
    {
        "ID": "12345ABC",
        "start_location": [40.7128, -74.0060],   # New York, NY
        "end_location": [34.0522, -118.2437],    # Los Angeles, CA
        "current_location": [39.7392, -104.9903], # Denver, CO
        "receiver_name": "Alice Johnson",
        "shipping_person": "John Doe",
        "status": "In Transit"
    },
    {
        "ID": "67890XYZ",
        "start_location": [41.8781, -87.6298],   # Chicago, IL
        "end_location": [29.7604, -95.3698],     # Houston, TX
        "current_location": [33.4484, -112.0740], # Phoenix, AZ
        "receiver_name": "Bob Smith",
        "shipping_person": "Jane Doe",
        "status": "Departed from Facility"
    },
    {
        "ID": "24680DEF",
        "start_location": [34.0522, -118.2437],  # Los Angeles, CA
        "end_location": [36.1699, -115.1398],    # Las Vegas, NV
        "current_location": [34.0522, -118.2437], # Los Angeles, CA
        "receiver_name": "Charlie Brown",
        "shipping_person": "Jim Ray",
        "status": "Arrived at Facility"
    },
    {
        "ID": "13579GHI",
        "start_location": [25.7617, -80.1918],   # Miami, FL
        "end_location": [35.2271, -80.8431],     # Charlotte, NC
        "current_location": [32.7765, -79.9311], # Charleston, SC
        "receiver_name": "David Lee",
        "shipping_person": "Sam Miles",
        "status": "Out for Delivery"
    },
    {
        "ID": "97531JKL",
        "start_location": [47.6062, -122.3321],  # Seattle, WA
        "end_location": [45.5152, -122.6784],    # Portland, OR
        "current_location": [46.8523, -121.7603], # Mount Rainier, WA
        "receiver_name": "Eve Green",
        "shipping_person": "Tom Grey",
        "status": "On Hold"
    },
    {
        "ID": "11223MNO",
        "start_location": [39.9526, -75.1652],   # Philadelphia, PA
        "end_location": [33.7490, -84.3880],     # Atlanta, GA
        "current_location": [38.9072, -77.0369], # Washington, D.C.
        "receiver_name": "Frank White",
        "shipping_person": "Mike Turner",
        "status": "In Transit"
    },
    {
        "ID": "44556PQR",
        "start_location": [37.7749, -122.4194],  # San Francisco, CA
        "end_location": [34.0522, -118.2437],    # Los Angeles, CA
        "current_location": [36.7783, -119.4179], # Fresno, CA
        "receiver_name": "Grace Black",
        "shipping_person": "Linda Howard",
        "status": "Departed from Facility"
    },
    {
        "ID": "77889STU",
        "start_location": [29.7604, -95.3698],   # Houston, TX
        "end_location": [32.7767, -96.7970],     # Dallas, TX
        "current_location": [31.9686, -99.9018], # Texas (central)
        "receiver_name": "Henry Brown",
        "shipping_person": "Chris Lee",
        "status": "Arrived at Facility"
    },
    {
        "ID": "99112VWX",
        "start_location": [42.3601, -71.0589],   # Boston, MA
        "end_location": [40.7128, -74.0060],     # New York, NY
        "current_location": [41.2033, -77.1945], # Pennsylvania
        "receiver_name": "Ivy Wilson",
        "shipping_person": "John Peters",
        "status": "Out for Delivery"
    },
    {
        "ID": "22334YZA",
        "start_location": [32.7157, -117.1611],  # San Diego, CA
        "end_location": [36.7783, -119.4179],    # California (central)
        "current_location": [33.7743, -118.0730], # Long Beach, CA
        "receiver_name": "Jack Quinn",
        "shipping_person": "Susan White",
        "status": "On Hold"
    },
    {
        "ID": "33445BCD",
        "start_location": [40.7128, -74.0060],   # New York, NY
        "end_location": [38.6270, -90.1994],     # St. Louis, MO
        "current_location": [39.1031, -84.5120], # Cincinnati, OH
        "receiver_name": "Karen Thomas",
        "shipping_person": "Alan Smith",
        "status": "In Transit"
    },
    {
        "ID": "55667EFG",
        "start_location": [34.0522, -118.2437],  # Los Angeles, CA
        "end_location": [36.1699, -115.1398],    # Las Vegas, NV
        "current_location": [35.4737, -120.7700], # Paso Robles, CA
        "receiver_name": "Liam Young",
        "shipping_person": "Olivia Lee",
        "status": "Departed from Facility"
    },
    {
        "ID": "66778HIJ",
        "start_location": [25.7617, -80.1918],   # Miami, FL
        "end_location": [35.2271, -80.8431],     # Charlotte, NC
        "current_location": [31.4505, -82.9507], # Georgia
        "receiver_name": "Mia King",
        "shipping_person": "Emma Green",
        "status": "Arrived at Facility"
    },
    {
        "ID": "78990KLM",
        "start_location": [47.6062, -122.3321],  # Seattle, WA
        "end_location": [45.5152, -122.6784],    # Portland, OR
        "current_location": [46.8523, -121.7603], # Mount Rainier, WA
        "receiver_name": "Noah Hill",
        "shipping_person": "Liam Brown",
        "status": "Out for Delivery"
    },
    {
        "ID": "89001NOP",
        "start_location": [39.9526, -75.1652],   # Philadelphia, PA
        "end_location": [33.7490, -84.3880],     # Atlanta, GA
        "current_location": [38.9072, -77.0369], # Washington, D.C.
        "receiver_name": "Olivia Davis",
        "shipping_person": "Sophia Clark",
        "status": "On Hold"
    },
    {
        "ID": "10112QRS",
        "start_location": [37.7749, -122.4194],  # San Francisco, CA
        "end_location": [34.0522, -118.2437],    # Los Angeles, CA
        "current_location": [36.7783, -119.4179], # Fresno, CA
        "receiver_name": "Paul Walker",
        "shipping_person": "James Lee",
        "status": "In Transit"
    },
    {
        "ID": "12132TUV",
        "start_location": [29.7604, -95.3698],   # Houston, TX
        "end_location": [32.7767, -96.7970],     # Dallas, TX
        "current_location": [31.9686, -99.9018], # Texas (central)
        "receiver_name": "Quinn Allen",
        "shipping_person": "Grace Brown",
        "status": "Departed from Facility"
    },
    {
        "ID": "14151WXY",
        "start_location": [42.3601, -71.0589],   # Boston, MA
        "end_location": [40.7128, -74.0060],     # New York, NY
        "current_location": [41.2033, -77.1945], # Pennsylvania
        "receiver_name": "Rose Turner",
        "shipping_person": "Ella Green",
        "status": "Arrived at Facility"
    },
    {
        "ID": "16171ZAB",
        "start_location": [32.7157, -117.1611],  # San Diego, CA
        "end_location": [36.7783, -119.4179],    # California (central)
        "current_location": [33.7743, -118.0730], # Long Beach, CA
        "receiver_name": "Sam Martin",
        "shipping_person": "Ella Taylor",
        "status": "Out for Delivery"
    },
    {
        "ID": "18182CDE",
        "start_location": [40.7128, -74.0060],   # New York, NY
        "end_location": [38.6270, -90.1994],     # St. Louis, MO
        "current_location": [39.1031, -84.5120], # Cincinnati, OH
        "receiver_name": "Tina Sanders",
        "shipping_person": "Charlie Lee",
        "status": "On Hold"
    }
]


In [26]:
load_dotenv()
genai.configure(api_key=os.environ["GEMINI_TOKEN"])

In [39]:
gen_config = genai.GenerationConfig(
    temperature=0.4,
)

In [40]:
model = genai.GenerativeModel("gemini-1.5-flash", generation_config= gen_config)

In [49]:
history = []

In [75]:
context = input("Please enter your query: ")


prompt = f"""
System Prompt: You are an assistant designed to extract specific shipping information from user queries related to a database with the following fields for each shipping instance: 
- ID
- Start Location
- End Location
- Current Location
- Receiver Name
- Shipping Person
- Status

Your task is to identify and extract values or parameters needed from the user's question. Respond only in JSON format.

### User Query:
{context}

### Examples:
1. **User Question**: What's the current status of the shipment with ID 12345XYZ?  
   **Output**: `{{"ID": "12345XYZ"}}`

2. **User Question**: Where is the package for Alice Brown?  
   **Output**: `{{"receiver_name": "Alice Brown"}}`

3. **User Question**: Where is the package for Michael Smith currently?  
   **Output**: `{{"receiver_name": "Michael Smith"}}`

4. **User Question**: What is the current location of the shipment with ID 98765ABC?  
   **Output**: `{{"ID": "98765ABC", "current_location": "current"}}`

### Note:
Ensure the output is a well-structured JSON object containing only the relevant fields identified from the user’s question.
"""


print(context)


what is shippment information of Sam Martin


In [76]:
history.append({"role":"user", "parts":prompt})
res = model.generate_content(history, generation_config=gen_config)
history.append({"role":"model", "parts":res.text})

In [77]:
res.text

'```json\n{"receiver_name": "Sam Martin"}\n```'

In [78]:
def json_response(res):
    response = res.text.strip("```json\n").strip("\n```")
    re = json.loads(response)
    result = {
        "receiver_name": None,
        "ID": None
    }
    if "receiver_name" in re:
        result["receiver_name"] = re["receiver_name"]
    elif "ID" not in re:
        result["ID"] = re["ID"]
    return result

In [79]:
re = json_response(res)
re

{'receiver_name': 'Sam Martin', 'ID': None}

In [85]:
def look_up_data(extracted_data):
    if not extracted_data["receiver_name"] and not extracted_data["ID"]:
        return "Not enough information to search for shipping data"
    receiver_name = extracted_data["receiver_name"]
    shipment_id = extracted_data["ID"]
    results = []
    for shipment in shipping_data:
        if (receiver_name is not None and shipment["receiver_name"] == receiver_name) or \
           (shipment_id is not None and shipment["ID"] == shipment_id):
            results.append(shipment)

    if results:
        return results
    else:
        return "No matching shipping data found"


shippment_information = look_up_data(re)[0]
shippment_information

{'ID': '16171ZAB',
 'start_location': [32.7157, -117.1611],
 'end_location': [36.7783, -119.4179],
 'current_location': [33.7743, -118.073],
 'receiver_name': 'Sam Martin',
 'shipping_person': 'Ella Taylor',
 'status': 'Out for Delivery'}

In [87]:
response_prompt = f"""
with data below answer the user query:
Data:
ID: {shippment_information["ID"]}
start_location: {shippment_information["start_location"]}
end_location: {shippment_information["end_location"]}
current_location: {shippment_information["current_location"]}
receiver_name: {shippment_information["receiver_name"]}
shipping_person: {shippment_information["shipping_person"]}
status: {shippment_information["status"]}

User query: {context}


"""



print(response_prompt)


with data below answer the user query:
Data:
ID: 16171ZAB
start_location: [32.7157, -117.1611]
end_location: [36.7783, -119.4179]
current_location: [33.7743, -118.073]
receiver_name: Sam Martin
shipping_person: Ella Taylor
status: Out for Delivery

User query: what is shippment information of Sam Martin





In [88]:
res = model.generate_content(response_prompt, generation_config=gen_config)
res.text

'Here\'s the shipment information for Sam Martin based on the provided data:\n\n**Shipment Details:**\n\n* **ID:** 16171ZAB\n* **Receiver Name:** Sam Martin\n* **Start Location:** [32.7157, -117.1611] \n* **End Location:** [36.7783, -119.4179]\n* **Current Location:** [33.7743, -118.073] \n* **Shipping Person:** Ella Taylor\n* **Status:** Out for Delivery\n\n**Explanation:**\n\nThe data provides information about a shipment, including the ID, receiver name, start and end locations, current location, shipping person, and status.  By filtering the data based on the receiver name "Sam Martin", we can extract the relevant shipment information. \n'